In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [103]:
class Neuron:
    def __init__(self, nin):
        self.w = torch.randn(nin).requires_grad_(True)
        self.b = torch.randn(1).requires_grad_(True)
    def __call__(self, x):
        return torch.tanh(x @ self.w + self.b)
    def parameters(self):
        return [self.w] + [self.b]
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self, x):
        return torch.cat([n(x) for n in self.neurons])
    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]
class MLP:
    def __init__(self, nin, nout):
        sz = [nin] + nout
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nout))]
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [104]:
x = torch.tensor([2, 3, -1],dtype=torch.float32)
n = MLP(3, [4,4,1])
n(x)

tensor([0.9091], grad_fn=<CatBackward0>)

In [237]:
xs = [
    [2,3,-1],
    [3,-1,0],
    [0,1,-1],
    [1,-1,2],
    ]
ys = [1,1,0,0]
xs = torch.tensor(xs,dtype=torch.float32)
ys = torch.tensor(ys,dtype=torch.float32)
ypred = [n(x) for x in xs]
ypred = torch.cat(ypred)
ypred

tensor([ 0.1882,  0.8724,  0.2605, -0.3821], grad_fn=<CatBackward0>)

In [563]:
loss.backward()

In [564]:
for p in n.parameters():
    if p.grad is not None:
        p.data += -0.01 * p.grad


In [565]:
ypred = [n(x) for x in xs]
loss = sum([(yout-ygt)**2 for (yout,ygt) in zip(ypred,ys)])
loss

tensor([0.0021], grad_fn=<AddBackward0>)

In [544]:
ypred

[tensor([0.9839], grad_fn=<CatBackward0>),
 tensor([0.9959], grad_fn=<CatBackward0>),
 tensor([-0.0446], grad_fn=<CatBackward0>),
 tensor([-0.0771], grad_fn=<CatBackward0>)]